# Demo

In [1]:
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.metrics import mean_squared_error


from algorithms import matching_pursuit, orthogonal_matching_pursuit, weak_orthogonal_matching_pursuit

## 1. Generate Data

In [3]:
def generate_gaussian_noises_dict(N, d):
    gaussian_noises = np.random.normal(size=(d, N))
    norms = np.linalg.norm(gaussian_noises, axis=0, keepdims=True)
    # Create unit-norm vectors
    unit_vectors = gaussian_noises / norms
    return unit_vectors

def generate_sparse_response(gaussian_matrix, m):
    indices = np.random.choice(gaussian_matrix.shape[1], size=m, replace=False)
    selected_vectors = gaussian_matrix[:, indices]
    coefficients = np.random.normal(size=(m, 1))  # random coefficients for each selected vector
    y = selected_vectors @ coefficients
    return y, indices, coefficients

def generate_perturbed_response(y, noise_level):
    norm_y = np.linalg.norm(y)
    noise = np.random.normal(size=y.shape, scale=norm_y * noise_level)
    y_perturbed = y + noise
    return y_perturbed

In [4]:
# Use the functions to generate a Gaussian noise matrix and a sparse response
np.random.seed(0)
N = 100000
d = 300
m = 2
dictionary = generate_gaussian_noises_dict(N, d)
y, indices, coefficients = generate_sparse_response(dictionary, m)
noise_scale = 0.05
y_perturbed = generate_perturbed_response(y, noise_scale)

Check coefs

In [5]:
coefficients

array([[0.34249941],
       [1.7701584 ]])

Check indices

In [6]:
indices

array([16274, 83612])

## Algorithms

### MP

In [7]:
MP_residual, MP_indices, MP_coefficients = matching_pursuit(y, dictionary, 2*m)

MP_residual_perturb, MP_indices_perturb, MP_coefficients_perturb = matching_pursuit(y_perturbed, dictionary, 2*m)

### OMP

In [8]:
OMP_residual, OMP_indices, OMP_coefficients = orthogonal_matching_pursuit(y, dictionary, 2*m)

In [9]:
OMP_residual_perturb, OMP_indices_perturb, OMP_coefficients_perturb = orthogonal_matching_pursuit(y_perturbed, dictionary, 2*m)

### WOMP

In [10]:
WOMP_residual, WOMP_indices, WOMP_coefficients = weak_orthogonal_matching_pursuit(y, dictionary, 0.001)

WOMP_residual_perturb, WOMP_indices_perturb, WOMP_coefficients_perturb = weak_orthogonal_matching_pursuit(y_perturbed, dictionary, 0.001)

# Lasso

In [13]:
ALPHA = 0.001

# Not perturbed signal

lasso_signal = Lasso(alpha=ALPHA, fit_intercept=False, max_iter=10000, tol=0.0001)
lasso_signal.fit(dictionary, y)

coef = lasso_signal.coef_
nonzero_indices = np.nonzero(coef)[0]
nonzero_coef = coef[nonzero_indices]

print("Non-zero coefficients in Lasso: ", len(nonzero_coef))
print("Non-zero coefficients index are: ", nonzero_indices)
print("Non-zero coefficients are: ", nonzero_coef)


print('-----------------------------------------')

# Perturbed signal

lasso_signal_purturbed = Lasso(alpha=ALPHA, fit_intercept=False, max_iter=10000, tol=0.0001)
lasso_signal_purturbed.fit(dictionary, y_perturbed)

coef_purturbed = lasso_signal_purturbed.coef_
nonzero_indices_purturbed = np.nonzero(coef_purturbed)[0]
nonzero_coef_purturbed = coef[nonzero_indices_purturbed]

print("Non-zero coefficients in Lasso: ", len(nonzero_coef_purturbed))
print("Non-zero coefficients index are: ", nonzero_indices_purturbed)
print("Non-zero coefficients are: ", nonzero_coef_purturbed)

Non-zero coefficients in Lasso:  2
Non-zero coefficients index are:  [16274 83612]
Non-zero coefficients are:  [0.049707   1.47736601]
-----------------------------------------
Non-zero coefficients in Lasso:  29
Non-zero coefficients index are:  [  638  2217  2251  3201 11150 13331 19936 20267 20635 24049 26199 27670
 33398 38994 42414 43615 57263 59672 61548 70167 71521 73017 75459 75990
 78164 83612 90424 90459 98641]
Non-zero coefficients are:  [-0.          0.          0.         -0.         -0.         -0.
 -0.          0.         -0.         -0.          0.          0.
 -0.          0.          0.         -0.         -0.          0.
  0.          0.          0.         -0.         -0.         -0.
  0.          1.47736601 -0.          0.          0.        ]
